In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [3]:
URM_all = load_urm()

#URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)
URM_train = sps.load_npz('Dataset/Split/URM_train.npz')
URM_test =  sps.load_npz('Dataset/Split/URM_test.npz')
URM_validation = sps.load_npz('Dataset/Split/URM_validation.npz')

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1515 ( 3.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 975 ( 2.3%) Users that have less than 1 test interactions


In [4]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

#fit the IALS model tuned in previous experiments
mf_ials = IALSRecommender(URM_all)
mf_ials.fit(num_factors = 53,
            confidence_scaling= 'linear',
            alpha = 1.0,
            epsilon = 8.071768337775882,
            reg = 1e-05,
            epochs = 45)

IALSRecommender: Epoch 1 of 45. Elapsed time 23.35 sec
IALSRecommender: Epoch 2 of 45. Elapsed time 47.72 sec
IALSRecommender: Epoch 3 of 45. Elapsed time 1.15 min
IALSRecommender: Epoch 4 of 45. Elapsed time 1.53 min
IALSRecommender: Epoch 5 of 45. Elapsed time 1.91 min
IALSRecommender: Epoch 6 of 45. Elapsed time 2.29 min
IALSRecommender: Epoch 7 of 45. Elapsed time 2.65 min
IALSRecommender: Epoch 8 of 45. Elapsed time 3.02 min
IALSRecommender: Epoch 9 of 45. Elapsed time 3.40 min
IALSRecommender: Epoch 10 of 45. Elapsed time 3.74 min
IALSRecommender: Epoch 11 of 45. Elapsed time 4.13 min
IALSRecommender: Epoch 12 of 45. Elapsed time 4.50 min
IALSRecommender: Epoch 13 of 45. Elapsed time 4.85 min
IALSRecommender: Epoch 14 of 45. Elapsed time 5.23 min
IALSRecommender: Epoch 15 of 45. Elapsed time 5.60 min
IALSRecommender: Epoch 16 of 45. Elapsed time 5.97 min
IALSRecommender: Epoch 17 of 45. Elapsed time 6.34 min
IALSRecommender: Epoch 18 of 45. Elapsed time 6.72 min
IALSRecommender: 

In [6]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

# create a Slim elasticnet object
slim_en = SLIMElasticNetRecommender(URM_all)
slim_en.fit(topK=799, l1_ratio=0.006782112530625445, alpha=0.0023657508503917664)

SLIMElasticNetRecommender: Processed 2363 ( 9.6%) in 5.00 min. Items per second: 7.87
SLIMElasticNetRecommender: Processed 4854 (19.8%) in 10.00 min. Items per second: 8.09
SLIMElasticNetRecommender: Processed 7425 (30.3%) in 15.00 min. Items per second: 8.25
SLIMElasticNetRecommender: Processed 9913 (40.4%) in 20.00 min. Items per second: 8.26
SLIMElasticNetRecommender: Processed 12371 (50.5%) in 25.00 min. Items per second: 8.25
SLIMElasticNetRecommender: Processed 14883 (60.7%) in 30.01 min. Items per second: 8.27
SLIMElasticNetRecommender: Processed 17410 (71.0%) in 35.01 min. Items per second: 8.29
SLIMElasticNetRecommender: Processed 19975 (81.5%) in 40.01 min. Items per second: 8.32
SLIMElasticNetRecommender: Processed 22541 (92.0%) in 45.01 min. Items per second: 8.35
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 48.84 min. Items per second: 8.36


In [8]:
# create a recommender object which performs the hybridation of the 2 models
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
    def fit(self, alpha = 0.5):
        self.alpha = alpha      


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [9]:
for alpha in np.arange(0,1,0.1):
    scoreshybridrecommender = ScoresHybridRecommender(URM_train, slim_en, mf_ials)
    scoreshybridrecommender.fit(alpha = alpha)
    print("Hybrid model with alpha weight:")
    print(alpha)
    result_df, _ = evaluator_validation.evaluateRecommender(scoreshybridrecommender)
    print("MAP: {}".format(result_df.loc[10]["MAP"]))

Hybrid model with alpha weight:
0.0
EvaluatorHoldout: Processed 40114 (100.0%) in 1.28 min. Users per second: 521
MAP: 0.04741515055108335
Hybrid model with alpha weight:
0.1
EvaluatorHoldout: Processed 40114 (100.0%) in 1.27 min. Users per second: 526
MAP: 0.04705265984008596
Hybrid model with alpha weight:
0.2
EvaluatorHoldout: Processed 40114 (100.0%) in 1.27 min. Users per second: 526
MAP: 0.046660727244808096
Hybrid model with alpha weight:
0.30000000000000004
EvaluatorHoldout: Processed 40114 (100.0%) in 1.28 min. Users per second: 523
MAP: 0.046104198273012875
Hybrid model with alpha weight:
0.4
EvaluatorHoldout: Processed 40114 (100.0%) in 1.29 min. Users per second: 518
MAP: 0.04527013092052322
Hybrid model with alpha weight:
0.5
EvaluatorHoldout: Processed 40114 (100.0%) in 1.29 min. Users per second: 518
MAP: 0.044089609493890985
Hybrid model with alpha weight:
0.6000000000000001
EvaluatorHoldout: Processed 40114 (100.0%) in 1.28 min. Users per second: 523
MAP: 0.04229577845

In [ ]:
recommender = ScoresHybridRecommender(URM_all, slim_en, mf_ials)
recommender.fit(norm=1, alpha = 0.1)

## Submissions

In [ ]:
test_users = pd.read_csv('Dataset/data_target_users_test.csv')

In [ ]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10))

In [ ]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('Submissions\Submission_SLIM_EN_IALS_Scores.csv', index=False)